<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="Skills Network Logo">
    </a>
</p>


# Generative AI for Querying Databases

In this lab you will leverage Generative AI platforms to create optimized queries for your data

You will need to download the Heart Disease datast from UC Irvine Machine Learning repository and setp a sqllite database to run queries

## Instal and/or import required libraries

In [67]:
# Install the UC Irvine Machine Learning Repository
!pip install ucimlrepo 

In [68]:
import pandas as pd
import csv, sqlite3

### Importing the dataset:
Import into Python using the UC Irvine Machine Learning library

In [70]:
from ucimlrepo import fetch_ucirepo  
# fetch dataset 
heart_disease = fetch_ucirepo(id=45) 
  
# data (as pandas dataframes) 
X = heart_disease.data.features 
y = heart_disease.data.targets 
  
# metadata 
# print(heart_disease.metadata) 
  
# variable information 
# print(heart_disease.variables)

In [156]:
# Concatenate the features and target dataframes into 1 table for use in the sqlite database
df = pd.DataFrame(data=heart_disease.data.original)
df.rename(columns={'num':'target'}, inplace=True)
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2
2,67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,45,1,1,110,264,0,0,132,0,1.2,2,0.0,7.0,1
299,68,1,4,144,193,1,0,141,0,3.4,2,2.0,7.0,2
300,57,1,4,130,131,0,0,115,1,1.2,2,1.0,7.0,3
301,57,0,2,130,236,0,2,174,0,0.0,2,1.0,3.0,1


In [241]:
heart_disease.metadata.additional_info.variable_info

'Only 14 attributes used:\r\n      1. #3  (age)       \r\n      2. #4  (sex)       \r\n      3. #9  (cp)        \r\n      4. #10 (trestbps)  \r\n      5. #12 (chol)      \r\n      6. #16 (fbs)       \r\n      7. #19 (restecg)   \r\n      8. #32 (thalach)   \r\n      9. #38 (exang)     \r\n      10. #40 (oldpeak)   \r\n      11. #41 (slope)     \r\n      12. #44 (ca)        \r\n      13. #51 (thal)      \r\n      14. #58 (num)       (the predicted attribute)\r\n\r\nComplete attribute documentation:\r\n      1 id: patient identification number\r\n      2 ccf: social security number (I replaced this with a dummy value of 0)\r\n      3 age: age in years\r\n      4 sex: sex (1 = male; 0 = female)\r\n      5 painloc: chest pain location (1 = substernal; 0 = otherwise)\r\n      6 painexer (1 = provoked by exertion; 0 = otherwise)\r\n      7 relrest (1 = relieved after rest; 0 = otherwise)\r\n      8 pncaden (sum of 5, 6, and 7)\r\n      9 cp: chest pain type\r\n        -- Value 1: typical ang

In [158]:
# Load the sql extention and create database, connector, and cursor, then read table into database
%load_ext sql
con = sqlite3.connect("heart_disease.db")
cur = con.cursor()
%sql sqlite:///heart_disease.db
df.to_sql("heart_disease_prediction_dataset", con, if_exists='replace', index=False,method="multi")

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


303

In [160]:
%%sql
SELECT 
    MIN(age) AS min_age,
    MAX(age) AS max_age,
    AVG(age) AS avg_age
FROM 
    heart_disease_prediction_dataset;

 * sqlite:///heart_disease.db
Done.


min_age,max_age,avg_age
29,77,54.43894389438944


In [162]:
%%sql
SELECT 
    MIN(age) AS min_age,
    MAX(age) AS max_age,
    AVG(age) AS avg_age
FROM 
    heart_disease_prediction_dataset;

 * sqlite:///heart_disease.db
Done.


min_age,max_age,avg_age
29,77,54.43894389438944


In [164]:
%%sql
SELECT 
    cp,
    COUNT(*) AS frequency
FROM 
    heart_disease_prediction_dataset
GROUP BY 
    cp;

 * sqlite:///heart_disease.db
Done.


cp,frequency
1,23
2,50
3,86
4,144


In [166]:
%%sql
SELECT 
    MIN(chol) AS min_cholesterol,
    MAX(chol) AS max_cholesterol
FROM 
    heart_disease_prediction_dataset;

 * sqlite:///heart_disease.db
Done.


min_cholesterol,max_cholesterol
126,564


In [168]:
%%sql
SELECT 
    sex,
    MIN(age) AS min_age,
    MAX(age) AS max_age
FROM 
    heart_disease_prediction_dataset
GROUP BY 
    sex;

 * sqlite:///heart_disease.db
Done.


sex,min_age,max_age
0,34,76
1,29,77


In [171]:
%%sql
SELECT 
    CASE
        WHEN age BETWEEN 20 AND 30 THEN '20-30'
        WHEN age BETWEEN 31 AND 40 THEN '31-40'
        WHEN age BETWEEN 41 AND 50 THEN '41-50'
        WHEN age BETWEEN 51 AND 60 THEN '51-60'
        WHEN age BETWEEN 61 AND 70 THEN '61-70'
        ELSE 'Above 70'
    END AS age_group,
    target,
    COUNT(*) AS frequency
FROM 
    heart_disease_prediction_dataset
GROUP BY 
    age_group, target;

 * sqlite:///heart_disease.db
Done.


age_group,target,frequency
20-30,0,1
31-40,0,11
31-40,1,3
31-40,3,2
31-40,4,1
41-50,0,53
41-50,1,11
41-50,2,5
41-50,3,6
41-50,4,1


In [173]:
%%sql
SELECT 
    CASE
        WHEN age BETWEEN 30 AND 40 THEN '30-40'
        WHEN age BETWEEN 41 AND 50 THEN '41-50'
        WHEN age BETWEEN 51 AND 60 THEN '51-60'
        WHEN age BETWEEN 61 AND 70 THEN '61-70'
        ELSE 'Above 70'
    END AS age_group,
    MAX(thalach) AS max_heart_rate
FROM 
    heart_disease_prediction_dataset
GROUP BY 
    age_group;

 * sqlite:///heart_disease.db
Done.


age_group,max_heart_rate
30-40,192
41-50,194
51-60,195
61-70,179
Above 70,202


In [182]:
%%sql
SELECT 
    (CAST(SUM(CASE WHEN fbs = 1 THEN 1 ELSE 0 END) AS FLOAT) / COUNT(*)) * 100 AS percentage
FROM 
    heart_disease_prediction_dataset;

 * sqlite:///heart_disease.db
Done.


percentage
14.85148514851485


In [204]:
%%sql
SELECT 
    (CAST(SUM(CASE WHEN restecg != 0 THEN 1 ELSE 0 END) AS FLOAT) / SUM(CASE WHEN restecg = 0 THEN 1 ELSE 0 END)) AS ratio
FROM 
    heart_disease_prediction_dataset;

 * sqlite:///heart_disease.db
Done.


ratio
1.0066225165562914


In [246]:
%%sql
SELECT 
    COUNT(*) AS num_patients
FROM 
    heart_disease_prediction_dataset
WHERE 
    thal = 7;

 * sqlite:///heart_disease.db
Done.


num_patients
117


In [250]:
%%sql
SELECT 
    AVG(age) AS average_age
FROM 
    heart_disease_prediction_dataset
WHERE 
    cp > 0;

 * sqlite:///heart_disease.db
Done.


average_age
54.43894389438944


In [255]:
%%sql
SELECT 
    ca,
    COUNT(*) AS num_patients
FROM 
    heart_disease_prediction_dataset
GROUP BY 
    ca;

 * sqlite:///heart_disease.db
Done.


ca,num_patients
None,4
0.0,176
1.0,65
2.0,38
3.0,20
